In [1]:
import os
import numpy as np
import re
import string

In [2]:
def log_progress(sequence, every=None, size=None, name='Items'):
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display

    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = int(size / 200)     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)

    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{name}: {index} / ?'.format(
                        name=name,
                        index=index
                    )
                else:
                    progress.value = index
                    label.value = u'{name}: {index} / {size}'.format(
                        name=name,
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = "{name}: {index}".format(
            name=name,
            index=str(index or '?')
        )

In [3]:
FILE_UNIGRAMS = '1grams-3.txt'
FILE_TEXT = 'steve.txt'
EPSILON = 1e-20

In [4]:
EXCLUDE_SYMBOLS_STR = u''.join(['№', '«', 'ђ', '°', '±', '‚', 'ћ', '‰', '…', '»', 'ѓ', 'µ', '·', 'ґ', 'њ', 'ї', 'џ', 'є',
                                '‹', '‡', '†', '¶', 'ќ', '€', '“', 'ў', '§', '„', '”', '\ufeff', '’', 'љ', '›', '•', '—',
                                '‘', '\x7f', '\xad', '¤', '\xa0'])

In [5]:
txt = open(FILE_TEXT, 'r', encoding="utf8").read()

In [6]:
txt

'Первая история – про соединение точек\n\nЯ бросил учебу в Reed college через полгода с момента поступления, но продолжал ходить на лекции и жить в студгородке еще 18 месяцев, пока не забросил это дело окончательно. Так почему я бросил учебу?\n\nЭта история началась до моего рождения. Моя биологическая мать, молодая незамужняя аспирантка, решила отдать меня на усыновление. Ей очень хотелось, чтобы меня усыновили люди с высшим образованием. И все было готово для того, чтобы меня взяли на воспитание в семью некоего юриста. Но к моменту моего рождения юрист и его жена вдруг решили, что на самом деле им нужна девочка, а не мальчик. Так что моим будущим родителям, которые были следующими в очереди, позвонили среди ночи с вопросом: «У нас есть внеплановый ребенок. Мальчик. Возьмете его?» И они ответили «Конечно». Позже моя биологическая мать узнала, что моя реальная мать не заканчивала никакого колледжа, и что мой отец не закончил даже средней школы. Она отказалась подписать окончательные бу

In [7]:
all_words_count = 0
with open(FILE_UNIGRAMS, 'r', encoding="utf8") as f:
    for line in f:
        split = line.strip().split('\t')
        try:
            freq, word = split
            all_words_count += float(freq)
        except:
            print(split)
all_words_count

190827174.0

In [8]:
frequencies = {}
with open(FILE_UNIGRAMS, 'r', encoding="utf8") as f:
    for line in f:
        split = line.strip().split('\t')
        try:
            freq, word = split
        except:
            print(split)
        frequencies[word] = float(freq) / all_words_count

In [9]:
len(frequencies)

1054210

In [10]:
frequencies

{'и': 0.03579138052948371,
 'в': 0.027198511046440377,
 'не': 0.01658516412342825,
 'на': 0.01430199872896509,
 'с': 0.010818207683566073,
 'что': 0.010817002404489835,
 'как': 0.006007441057634695,
 'я': 0.005723382981084235,
 'к': 0.005482929805374574,
 'он': 0.0054536834465724465,
 'по': 0.005088012255529184,
 'а': 0.005042940058421658,
 'его': 0.004875783571578752,
 'из': 0.004119235135767404,
 'это': 0.0038231923929240813,
 'от': 0.0037234214871305488,
 'все': 0.003645639063962662,
 'за': 0.0036192120101301716,
 'у': 0.0035254465383425947,
 'же': 0.003484194551872366,
 'то': 0.0034644489363972867,
 'В': 0.003419088520380226,
 'но': 0.0033440625180562594,
 'о': 0.003218178979058821,
 'И': 0.003074761249674011,
 'А': 0.0030345154092152513,
 'было': 0.003021011043217566,
 'для': 0.0028352408551624835,
 'так': 0.0028010266504287277,
 'Я': 0.0026199098876766892,
 'бы': 0.002544511820942231,
 'только': 0.00242420400775835,
 'был': 0.002252436018362877,
 'ее': 0.0022075000701944052,
 'он

In [11]:
frequencies['и']

0.03579138052948371

In [12]:
regex_puncts = re.compile('[%s]' % re.escape(string.punctuation))
regex_symbs = re.compile('[%s]' % re.escape(EXCLUDE_SYMBOLS_STR))
regex_digits = re.compile('[%s]' % re.escape(string.digits))
regex_spaces = re.compile('[%s]' % string.printable + string.whitespace)

In [13]:
txt_prep = regex_puncts.sub('', txt)

In [14]:
txt_prep = regex_symbs.sub('', txt_prep)

In [15]:
txt_prep = regex_digits.sub('', txt_prep)

In [16]:
txt_prep = regex_spaces.sub('', txt_prep)

In [17]:
txt_prep = txt_prep.strip().strip('\t').replace('\n', ' ').lower()

In [18]:
txt_prep

'первая история – про соединение точек  я бросил учебу в reed college через полгода с момента поступления но продолжал ходить на лекции и жить в студгородке еще  месяцев пока не забросил это дело окончательно так почему я бросил учебу  эта история началась до моего рождения моя биологическая мать молодая незамужняя аспирантка решила отдать меня на усыновление ей очень хотелось чтобы меня усыновили люди с высшим образованием и все было готово для того чтобы меня взяли на воспитание в семью некоего юриста но к моменту моего рождения юрист и его жена вдруг решили что на самом деле им нужна девочка а не мальчик так что моим будущим родителям которые были следующими в очереди позвонили среди ночи с вопросом у нас есть внеплановый ребенок мальчик возьмете его и они ответили конечно позже моя биологическая мать узнала что моя реальная мать не заканчивала никакого колледжа и что мой отец не закончил даже средней школы она отказалась подписать окончательные бумаги на усыновление лишь несколько 

In [19]:
words = txt_prep.split()

print(len(words))

1938


In [20]:
frequencies_empirical = {}

for word in log_progress(words):
    if frequencies_empirical.get(word, None) is None:
        frequencies_empirical[word] = 1
    else:
        frequencies_empirical[word] += 1

In [22]:
def perplexity(docs):
    docs = np.asarray(docs)
    out_of_dict = []
    
    tmp_sum_docs = 0.0
    N = 0.0
    for doc in log_progress(docs):
        tmp_sum_words = 0.0
        for word in log_progress(words):
            freq = frequencies.get(word, EPSILON)
            freq_empirical = frequencies_empirical.get(word, 0.0)
            N += freq_empirical
            if freq <= EPSILON:
                out_of_dict.append(word)
            tmp_sum_words += freq_empirical * np.log(freq)
        tmp_sum_docs += tmp_sum_words
        
    return np.exp(-tmp_sum_docs / N), out_of_dict

In [23]:
perp, ood = perplexity([words])

print(perp)

1104.1935125584232


In [24]:
print(np.unique(ood))

['apple' 'mcdonalds' 'pixar' 'reed' 'sansserif' 'serif' 'возняк'
 'выпускаетесь' 'гугла' 'дэвидом' 'какаято' 'какимто' 'какнибудь' 'лорин'
 'менло' 'нойсом' 'паккардом' 'поляроида' 'поновому' 'пошелтаки'
 'стенфорд' 'стив' 'стюарт' 'типографику' 'чьюто' 'эндоскоп' '–']
